# Failed `read_up_to` Attempt (for future reference)

My first objective in apparatus-4.ipynb was this:
- Switch from `read` to `read_up_to` for (hopefully) faster reading

I could not, for the life of me, get `read_up_to` working.  Consistently, after successfully reading 9 mini-batches of size 3, I'd get this error on the 10th attempt:
```
InvalidArgumentError: Shapes of all inputs must match: values[0].shape = [1] != values[2].shape = [3]
```
Trying to resolve this was taking too long--especially given how quickly things are deprecated and refactored in TensorFlow!  Maybe the whole mechanism will change by the time I really need to do that kind of optimization.  In the meantime, I don't want to forget this in case I need to try again to get it working.  For now, there are more important things to work on.

Anyway, without further ado, here's what happened...

## The Input Pipeline

In [1]:
import os
import tensorflow as tf
notebook_dir = 'bigcluster_parted/train'
g = tf.Graph()

### Mini-batch Design Matrix

Get the list of .csv files in the `notebook_dir`.

In [2]:
csv_file_list = [os.path.join(notebook_dir, x) for x in os.listdir(notebook_dir) if x.endswith('.csv')]

Add operations to the graph which prepare mini-batches.  **[NEW: Using `read_up_to` instead of `read`]**

In [3]:
BATCH_SIZE = 3
READ_UP_TO_BUFFER_SIZE = 5  # keep it small; this is just prototype exploration

with g.as_default():
    
    # Create a queue of the filenames
    filename_queue = tf.train.string_input_producer(csv_file_list)

    # (if there are lines of metadata at the top of the file, we can skip them using this parameter)
    reader = tf.TextLineReader(skip_header_lines=0)
    
    # Create graph operations to read a line from a CSV file (key is optional, but yields good info)
#     keys, lines_from_file = reader.read_up_to(filename_queue, num_records=READ_UP_TO_BUFFER_SIZE)
    keys, lines_from_file = reader.read_up_to(filename_queue, num_records=BATCH_SIZE)
    
    # Specify the default field values (in case they're missing) and their type
    record_defaults = [[0.5], [0.5], [tf.to_int32(0)]]
    
    # Parse the CSV strings into an example tensor ((x1,y1,w1,label1), (x2,y2,w2,label2), ..., (xn,yn,wn,labeln))
    examples = tf.decode_csv(lines_from_file, record_defaults=record_defaults)
    
    # Split the example into a feature vector and a label.
    # Add the extra 1 so we can apply bias via matrix multiplication (this extra 1 used to be right in the CSV file)
    features = tf.stack([examples[0], examples[1], tf.ones([len(examples)], tf.float32)], axis=1)
    labels = examples[-1]
    
    # Graph operations that produce a mini-batch (key batch is optional)
    key_batch, data_batch, label_batch = tf.train.shuffle_batch(
        [keys, features, labels],
        enqueue_many=True,
        batch_size=BATCH_SIZE, 
        capacity=25, 
        min_after_dequeue=2)

# g.finalize()

#### Quick check that the extra 1 is added properly

In [4]:
with tf.Session(graph=g) as s:
    s.run(tf.global_variables_initializer())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord, sess=s)
    
    for i in range(20):
        print("i = " + str(i))
        print(s.run([features, labels]))
#         print(s.run([labels]))
#         for j in range(data_batch.shape[0]):
#             print(data_batch[j], label_batch[j])

    coord.request_stop()
    coord.join(threads)

i = 0
[array([[-0.35833099, -0.211631  ,  1.        ],
       [-0.205256  ,  0.67395103,  1.        ],
       [ 0.660281  ,  1.32979   ,  1.        ]], dtype=float32), array([0, 0, 0], dtype=int32)]
i = 1
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Dequeue operation was cancelled
	 [[Node: ReaderReadUpToV2 = ReaderReadUpToV2[_device="/job:localhost/replica:0/task:0/cpu:0"](TextLineReaderV2, input_producer, ReaderReadUpToV2/num_records)]]
	 [[Node: DecodeCSV/_7 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/gpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=1, tensor_name="edge_12_DecodeCSV", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op 'ReaderReadUpToV2', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in

InvalidArgumentError: Shapes of all inputs must match: values[0].shape = [1] != values[2].shape = [3]
	 [[Node: stack = Pack[N=3, T=DT_FLOAT, axis=1, _device="/job:localhost/replica:0/task:0/gpu:0"](DecodeCSV/_13, DecodeCSV/_15, ones)]]

Caused by op 'stack', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/me/.local/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/me/.local/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/me/.local/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/home/me/.local/lib/python3.5/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/home/me/.loc